In [1]:
import pandana2

## Fetch network

pandana starts with a network (a DataFrame/GeoDataFrame of edges and a GeoDataFrame of nodes).  The easiest place to get a network is from osmnx.  The appropriate precision of network from osmnx is drive scale, but with all edges bidirectional and highways removed.

In [2]:
net = pandana2.PandanaNetwork.from_osmnx_local_streets_from_place_query("Oakland, CA")

In [4]:
net.edges

length  \
u           v                       
35718715    91846714     1007.270   
            35719057       66.828   
            843006075     571.686   
35718778    11379211154   197.852   
            35718864     1633.159   
...                           ...   
12685722200 595911953      81.736   
            53077725       44.788   
12685736901 53077727        7.525   
            283267099      82.432   
            12685722200    62.827   

                                                                  geometry  
u           v                                                               
35718715    91846714     LINESTRING (-122.24337 37.86098, -122.24328 37...  
            35719057     LINESTRING (-122.24337 37.86098, -122.24352 37...  
            843006075    LINESTRING (-122.24337 37.86098, -122.24337 37...  
35718778    11379211154  LINESTRING (-122.23083 37.86646, -122.23126 37...  
            35718864     LINESTRING (-122.23083 37.86646, -122.23077 37...  
...                                                                    ...  
12685722200 595911953    LINESTRING (-122.27200 37.79697, -122.27199 37...  
            53077725     LINESTRING (-122.27200 37.79697, -122.27172 37...  
12685736901 53077727     LINESTRING (-122.27265 37.79721, -122.27269 37...  
            283267099    LINESTRING (-122.27265 37.79721, -122.27217 37...  
            12685722200  LINESTRING (-122.27265 37.79721, -122.27200 37...  

[22312 rows x 2 columns]

In [5]:
net.nodes

,geometry
osmid,
35718715,POINT (-122.24337 37.86098)
35718778,POINT (-122.23083 37.86646)
35718864,POINT (-122.21786 37.86836)
35719057,POINT (-122.24405 37.86073)
35719067,POINT (-122.22193 37.86420)
...,...
12519070900,POINT (-122.25735 37.83788)
12625423764,POINT (-122.26173 37.81290)
12648792107,POINT (-122.27243 37.80718)


## Preprocess the network up to a maximum radius
This step runs a dijkstra on every input node up to the cutoff you pass to 'preprocess'.  The output is a dataframe with the shortest path edge weight for every from-to node combination where the minimum path is less than the cutff.

In [6]:
# preprocess to 1500 meters (edge weights are set to length by default, but travel times are handy when available)
net.preprocess(1500)
net.min_weights_df

,from,to,weight
0,35718715,35718715,0.00
1,35718715,35719057,66.83
3,35718715,843006075,571.69
2,35718715,91846714,1007.27
4,35718715,53100763,1274.03
...,...,...,...
1845291,12685736901,875338104,1473.08
1845296,12685736901,53033896,1478.00
1845299,12685736901,4169573493,1478.10
1845293,12685736901,3197662049,1480.18


## Get some data to aggregate
Here we load in a few hundred home sales obversvatons from the area we are analyzing.  We are going to aggregate the dollars per square foot metric.

In [7]:
df = pd.read_csv("../tests/data/redfin_2025-04-04-13-35-42.csv")
redfin_df = gpd.GeoDataFrame(
    df[["$/SQUARE FEET"]],
    geometry=gpd.points_from_xy(df.LONGITUDE, df.LATITUDE),
    crs="EPSG:4326",
)
redfin_df

,$/SQUARE FEET,geometry
0,838.0,POINT (-122.21295 37.80510)
1,1082.0,POINT (-122.24608 37.84365)
2,634.0,POINT (-122.26977 37.84511)
3,310.0,POINT (-122.17958 37.77725)
4,665.0,POINT (-122.26823 37.84393)
...,...,...
331,489.0,POINT (-122.25837 37.83088)
332,323.0,POINT (-122.21366 37.78334)
333,158.0,POINT (-122.16147 37.75856)
334,282.0,POINT (-122.23435 37.77754)


## Map each observation to it's nearest street intersection
This is a spatial join to the nodes DataFrame of the network

In [8]:
redfin_df["node_id"] = net.nearest_nodes(redfin_df)
redfin_df

,$/SQUARE FEET,geometry,node_id
0,838.0,POINT (-122.21295 37.80510),53041240
1,1082.0,POINT (-122.24608 37.84365),53077164
2,634.0,POINT (-122.26977 37.84511),53107934
3,310.0,POINT (-122.17958 37.77725),53103203
4,665.0,POINT (-122.26823 37.84393),53122194
...,...,...,...
331,489.0,POINT (-122.25837 37.83088),53095117
332,323.0,POINT (-122.21366 37.78334),53017889
333,158.0,POINT (-122.16147 37.75856),53105616
334,282.0,POINT (-122.23435 37.77754),53106534


## Do the aggregation
Just like that, we can do a weighted mean of all the observations where we apply a linear decay to the distance of each observation from each origin node so that nodes closer to the observed node are weighted higher.  NaN values mean that there are no observations within the requested distance.

In [9]:
nodes = net.nodes.copy()
nodes["average price/sqft"] = net.aggregate(
    values=pd.Series(redfin_df["$/SQUARE FEET"].values, index=redfin_df["node_id"]),
    decay_func=pandana2.LinearDecay(1500),
    aggregation="mean",
)
nodes["count"] = net.aggregate(
    values=pd.Series(1, index=redfin_df["node_id"]),
    decay_func=pandana2.NoDecay(1500),
    aggregation="sum",
)
nodes

,geometry,average price/sqft,count
osmid,,,
35718715,POINT (-122.24337 37.86098),NaN,NaN
35718778,POINT (-122.23083 37.86646),695.851186,4.0
35718864,POINT (-122.21786 37.86836),604.000000,1.0
35719057,POINT (-122.24405 37.86073),NaN,NaN
35719067,POINT (-122.22193 37.86420),585.688027,3.0
...,...,...,...
12519070900,POINT (-122.25735 37.83788),844.581826,27.0
12625423764,POINT (-122.26173 37.81290),314.459030,2.0
12648792107,POINT (-122.27243 37.80718),638.191425,2.0


In [10]:
import json
import os
from mapboxgl.utils import create_color_stops, df_to_geojson
from mapboxgl.viz import CircleViz
import warnings
warnings.filterwarnings("ignore")

In [11]:
colors = ['#edf8fb','#ccece6','#99d8c9','#66c2a4','#41ae76','#238b45','#005824']
token = os.environ['MAPBOX_TOKEN']
color_stops = create_color_stops([0, 200, 400, 600, 800, 1000, 1200], colors=colors)
circle_viz = CircleViz(
    json.loads(nodes.reset_index().round().to_json()),
    access_token=token,
    color_property='average price/sqft',
    color_stops=color_stops,
    center=[-122.2712, 37.8044],
    zoom=12,
    below_layer='waterway-label'
)
circle_viz.show()